In [1]:
import json
import os
import sys
import argparse
from collections import Counter, defaultdict
import numpy as np
import graph_tool.all as gt
import pandas as pd


src_dir = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path[0] = src_dir
from parser.support import ROLES, CREDITS
from parser.my_mongo_db_login import DB_LOGIN_INFO
import parser.support as support
import network.shift_graph_maker as sgm

In [2]:
src_dir

'/home/staff/junelee/Research-Project/movie_propagation/src'

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [8]:
with open('../data/raw_data/movies.json') as f:
    movie_file = f.read()
    movie_data = json.loads(movie_file)

In [14]:
role = 'producing'
role_key = role + "_gender_percentage"
all_movies = support.get_movies_df(role_key)
print('Got all_movies')

Loaded IMDb movies producing_gender_percentage
Got all_movies


In [15]:
gender_df = support.get_staff_df('producers')[['_id', 'female_count', 'first_movie', 'last_movie', 'gender']]

In [18]:
row = all_movies.iloc[0]
producers = [i[0] for i in row.producers]

In [21]:
for p in producers:
    print(gender_df[gender_df['_id']==p]['gender'].iloc[0])
    break

male


In [28]:
import random
def generate_seeds(df, num_seeds=2):
    """
    generate seed adopters based on the first person to come around
    """
    first_producers = [p[0] for p in df.producers.iloc[0]]

    # number of seeds are smaller than first participating producers
    if len(first_producers) >= num_seeds:
        seeds = random.sample(first_producers,num_seeds)
    else:
        i = 1
        seeds = first_producers
        while len(first_producers) < num_seeds:
            new_producers = [p[0] for p in df.producers.iloc[i]]
            new_seed = random.choice(new_producers)
            if new_seed not in first_producers:
                first_producers.append(new_seed)
    return seeds
    

In [29]:
generate_seeds(all_movies, 5)

['nm0289381', 'nm0909259', 'nm0912491', 'nm0288992', 'nm0768178']

In [37]:
generate_seeds(all_movies, 4)

['nm0289381', 'nm0909259', 'nm0912491', 'nm0768178']

In [31]:
all_movies.producers.iloc[0]

[['nm0289381', 'producer (uncredited)'],
 ['nm0909259', 'executive producer (uncredited)'],
 ['nm0912491', 'executive producer (uncredited)']]

In [32]:
all_movies.producers.iloc[1]

[['nm0288992', 'associate producer'], ['nm0768178', 'producer']]

In [44]:
gender_df[gender_df.gender == 'female']._id.tolist()

['nm0819749',
 'nm0103084',
 'nm0326291',
 'nm0916665',
 'nm0395116',
 'nm0681933',
 'nm0500334',
 'nm0954185',
 'nm1716114',
 'nm0546883',
 'nm1627417',
 'nm0727637',
 'nm0675470',
 'nm0000443',
 'nm0784287',
 'nm0306802',
 'nm0444915',
 'nm2095570',
 'nm0832217',
 'nm0490746',
 'nm0138287',
 'nm0859013',
 'nm0602160',
 'nm1466806',
 'nm0958384',
 'nm0382362',
 'nm1406112',
 'nm1181237',
 'nm0906811',
 'nm0410802',
 'nm0277663',
 'nm1749221',
 'nm0950207',
 'nm0842765',
 'nm1804707',
 'nm0410504',
 'nm1248435',
 'nm1085795',
 'nm0353658',
 'nm0193410',
 'nm0768930',
 'nm2654843',
 'nm0270682',
 'nm0714214',
 'nm2555285',
 'nm0000407',
 'nm0953307',
 'nm0935832',
 'nm1268417',
 'nm0326250',
 'nm0000054',
 'nm0000556',
 'nm0005476',
 'nm1343881',
 'nm0131837',
 'nm0081046',
 'nm2776434',
 'nm0589635',
 'nm0103061',
 'nm1853623',
 'nm0565565',
 'nm0698697',
 'nm1768448',
 'nm2336905',
 'nm2578668',
 'nm0237222',
 'nm1344809',
 'nm1681410',
 'nm3120816',
 'nm0588892',
 'nm1185591',
 'nm24